In [ ]:
source bioinf_intro_config.sh
rm -rf $IGV_DIR $STAR_OUT
mkdir -p $TRIMMED $STAR_OUT $IGV_DIR

# Run with shorter intron limit

IGV needs indices for the BAM files.  The index allows it to quickly load reads from different parts of the genome.

In [ ]:
for FASTQ in $RAW_FASTQS/27_MA_P_S38_L00[1]_R1_001.fastq.gz
    do
        FASTQ_BASE="$(basename ${FASTQ} '_001.fastq.gz')"
        echo "---------------- TRIMMING: $FASTQ_BASE ----------------"
        fastq-mcf \
            $MYINFO/neb_e7600_adapters.fasta \
            $RAW_FASTQS/${FASTQ_BASE}_001.fastq.gz \
            -q 20 -x 0.5 \
            -o $TRIMMED/${FASTQ_BASE}_001.trim.fastq.gz
        
        echo "---------------- MAPPING: $FASTQ_BASE ----------------"
        STAR \
            --runMode alignReads \
            --twopassMode None \
            --genomeDir $GENOME_DIR \
            --readFilesIn $TRIMMED/${FASTQ_BASE}_001.trim.fastq.gz \
            --readFilesCommand gunzip -c \
            --outFileNamePrefix ${STAR_OUT}/${FASTQ_BASE}_short_introns_ \
            --quantMode GeneCounts \
            --outSAMtype BAM SortedByCoordinate \
            --alignIntronMax 5000 \
            --outSJfilterIntronMaxVsReadN 500 1000 2000 \
            
        echo "---------------- INDEXING BAM: $FASTQ_BASE ----------------"
        samtools index ${STAR_OUT}/${FASTQ_BASE}_Aligned.sortedByCoord.out.bam
    done

In [ ]:
ls -ltr ${STAR_OUT}

#### Link Directory

In [ ]:
ln -s ${STAR_OUT}/*.bam* $GTF $FASTA $IGV_DIR

#### Tarring

In [ ]:
tar --dereference \
    --create \
    --gzip \
    --verbose \
    --file $CUROUT/stuff_for_igv.tgz \
    --directory $CUROUT \
    $(basename $IGV_DIR)